In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

In [2]:
def process_wind_file(filepath):
    df = pd.read_table(filepath, delim_whitespace=True)
    df = df.drop(index=df.index[0])
    df['#YY'] = df['#YY'].apply(int)
    #print((filepath[14:18]))
    df = df[df['#YY'] == int(filepath[20:24])]

    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df.groupby(['MM','DD','#YY']).mean().reset_index()
    df.rename(columns={'#YY': 'year', 'MM': 'month', 'DD': 'day'}, inplace=True)
    #add time to df which has datetime
    df['time'] = pd.to_datetime(df[['year', 'month', 'day']])
    df['time'] = df['time'].dt.date
    return df[['time','WSPD']]

df = process_wind_file("Data/Wind_46025/wind2012.txt")
df

/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_52057/2421531246.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_table(filepath, delim_whitespace=True)


,time,WSPD
0,2012-01-01,1.595833
1,2012-01-02,3.631818
2,2012-01-03,3.058333
3,2012-01-04,2.629167
4,2012-01-05,3.387500
...,...,...
140,2012-12-27,10.887500
141,2012-12-28,5.126087
142,2012-12-29,3.704167
143,2012-12-30,3.520833


In [3]:



# Generate a complete date range for 2010
all_dates = pd.date_range(start="2012-01-01", end="2012-12-31")

# Find the missing date
missing_date = all_dates.difference(df['time'])
print("Missing date:", missing_date)


Missing date: DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-19',
               ...
               '2012-11-14', '2012-11-15', '2012-11-16', '2012-11-17',
               '2012-11-18', '2012-11-19', '2012-11-20', '2012-11-21',
               '2012-11-22', '2012-11-23'],
              dtype='datetime64[ns]', length=221, freq=None)


In [4]:
fin_df = process_wind_file("Data/Wind_46025/wind2008.txt")
for i in range(2009, 2024, 1):
    #print(i)
    fin_df = pd.concat([fin_df, process_wind_file("Data/Wind_46025/wind" + str(i) + ".txt")])
fin_df

/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_52057/2421531246.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_table(filepath, delim_whitespace=True)
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_52057/2421531246.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_table(filepath, delim_whitespace=True)
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_52057/2421531246.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_table(filepath, delim_whitespace=True)
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_52057/2421531246.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated a

,time,WSPD
0,2008-01-01,3.687500
1,2008-01-02,3.758333
2,2008-01-03,2.050000
3,2008-01-04,4.012500
4,2008-01-05,6.833333
...,...,...
360,2023-12-27,2.990972
361,2023-12-28,2.878472
362,2023-12-29,2.540972
363,2023-12-30,4.835417


In [5]:
def to_str(dt):
    return dt.strftime('%m/%d/%Y')
len(np.unique(fin_df['time'].apply(to_str)))
fin_df['time'].apply(to_str)[-4:]

361    12/28/2023
362    12/29/2023
363    12/30/2023
364    12/31/2023
Name: time, dtype: object

In [6]:
len(np.unique(process_wind_file("Data/Wind_46025/wind" + str(2013) + ".txt")['time']))

/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_52057/2421531246.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_table(filepath, delim_whitespace=True)


365

In [7]:
process_wind_file("Data/Wind_46025/wind" + str(2012) + ".txt")

/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_52057/2421531246.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_table(filepath, delim_whitespace=True)


,time,WSPD
0,2012-01-01,1.595833
1,2012-01-02,3.631818
2,2012-01-03,3.058333
3,2012-01-04,2.629167
4,2012-01-05,3.387500
...,...,...
140,2012-12-27,10.887500
141,2012-12-28,5.126087
142,2012-12-29,3.704167
143,2012-12-30,3.520833


In [8]:
#For 46025
# Generate a complete date range for 2010
all_dates = pd.date_range(start="2008-03-05", end="2023-12-31")

# Find the missing date
missing_date = all_dates.difference(fin_df['time'])
print("Missing date:", missing_date)
print(f'There are {len(missing_date)} missing dates')

Missing date: DatetimeIndex(['2010-03-02', '2012-04-01', '2012-04-02', '2012-04-03',
               '2012-04-04', '2012-04-05', '2012-04-06', '2012-04-07',
               '2012-04-08', '2012-04-09',
               ...
               '2012-11-19', '2012-11-20', '2012-11-21', '2012-11-22',
               '2012-11-23', '2021-03-10', '2021-03-11', '2021-03-12',
               '2021-03-13', '2021-03-14'],
              dtype='datetime64[ns]', length=227, freq=None)
There are 227 missing dates


In [ ]:
#For 46025
# Generate a complete date range for 2010
all_dates = pd.date_range(start="2008-03-05", end="2023-12-31")

# Find the missing date
missing_date = all_dates.difference(fin_df['time'])
print("Missing date:", missing_date)
print(f'There are {len(missing_date)} missing dates')

Missing date: DatetimeIndex(['2010-03-02', '2012-04-01', '2012-04-02', '2012-04-03',
               '2012-04-04', '2012-04-05', '2012-04-06', '2012-04-07',
               '2012-04-08', '2012-04-09',
               ...
               '2012-11-19', '2012-11-20', '2012-11-21', '2012-11-22',
               '2012-11-23', '2021-03-10', '2021-03-11', '2021-03-12',
               '2021-03-13', '2021-03-14'],
              dtype='datetime64[ns]', length=227, freq=None)
There are 227 missing dates


In [9]:
#For La jolla

fin_df = process_wind_file("Data/Wind_Scrip/wind2008.txt")
for i in range(2009, 2024, 1):
    #print(i)
    fin_df = pd.concat([fin_df, process_wind_file("Data/Wind_Scrip/wind" + str(i) + ".txt")])
fin_df

/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_52057/2421531246.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_table(filepath, delim_whitespace=True)
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_52057/2421531246.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_table(filepath, delim_whitespace=True)
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_52057/2421531246.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_table(filepath, delim_whitespace=True)
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_52057/2421531246.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated a

,time,WSPD
0,2008-03-05,2.218182
1,2008-03-06,1.208333
2,2008-03-07,1.378947
3,2008-03-08,1.768421
4,2008-03-09,1.558333
...,...,...
360,2023-12-27,1.208333
361,2023-12-28,2.466667
362,2023-12-29,1.579167
363,2023-12-30,2.400000


In [10]:
all_dates = pd.date_range(start="2008-03-05", end="2023-12-31")

# Find the missing date
missing_date = all_dates.difference(fin_df['time'])
print("Missing date:", missing_date)
print(f'There are {len(missing_date)} missing dates')

Missing date: DatetimeIndex(['2008-03-16', '2008-04-02', '2008-04-03', '2008-04-04',
               '2008-04-05', '2008-04-06', '2008-04-07', '2008-04-08',
               '2008-04-09', '2008-04-10',
               ...
               '2017-08-10', '2019-10-13', '2021-03-10', '2021-03-11',
               '2021-03-12', '2021-03-13', '2021-03-14', '2021-03-15',
               '2021-03-16', '2022-11-09'],
              dtype='datetime64[ns]', length=491, freq=None)
There are 491 missing dates
